# Useful ADS Bits and pieces that made my life.. easy ish

![hearts](https://user-images.githubusercontent.com/30621653/76018449-f951de00-5f17-11ea-99a3-fb305583abfc.png)

# <u>Snippets</U>  (Extension)
>Quickly create re-useable code in simple steps
1. Open New SQL query window
2. Ctrl-Shift+P
3. Write some SQL you want to save
4. Type QE Boost and select QE Boost: Save New snippet
5. Give it a name starting SQL
6. Enter description
7. Then just Type in any query window sql.....name of the snippet
>

# Some more useful snippets

> ## Index stats

In [ ]:
IF (SELECT OBJECT_ID('Tempdb..#DatabaseAccess')) IS NOT NULL DROP TABLE #DatabaseAccess
CREATE TABLE #DatabaseAccess
(
     ServerName VARCHAR(50)
    ,DatabaseName VARCHAR(100)
    ,ReportDate DATETIME
    ,user_seeks BIGINT
    ,user_scans BIGINT
    ,user_lookups BIGINT
    ,reads BIGINT
    ,writes BIGINT
    ,ServerStartDate DATETIME

)

 
EXEC sp_MSFOREACHDB 'USE [?]

IF db_name() NOT IN (''TempDB'',''model'',''msdb'',''DBATools'',''master'')

INSERT INTO #DatabaseAccess
(
     ServerName
    ,DatabaseName
    ,ReportDate
    ,user_seeks
    ,user_scans
    ,user_lookups
    ,reads
    ,writes
    ,ServerStartDate

)


SELECT      
    @@ServerName AS ServerName,
    db_name() AS DatabaseName,
    CAST(GETDATE() as DATETIME) as ReportDate,
    SUM(user_seeks) AS user_seeks,   
    SUM(user_scans) AS user_scans,
    SUM(user_lookups) AS user_scans,
    reads=SUM(user_seeks + user_scans + user_lookups),         
    writes =  SUM(user_updates),   
    ServerStartDate = (select login_time from master.dbo.sysprocesses where spid =1)
FROM 
    sys.dm_db_index_usage_stats s        
INNER JOIN       
    sys.indexes i ON i.index_id = s.index_id AND s.object_id = i.object_id         
INNER JOIN       
    sys.objects o on s.object_id = o.object_id      
INNER JOIN       
    sys.schemas c on o.schema_id = c.schema_id  
INNER JOIN    
     sys.partitions AS p ON p.OBJECT_ID = i.OBJECT_ID AND p.index_id = i.index_id  
INNER JOIN   
     sys.allocation_units AS a ON a.container_id = p.partition_id  
WHERE       
        (OBJECTPROPERTY(s.object_id,''IsUserTable'') = 1  or OBJECTPROPERTY(s.object_id,''IsView'') = 1)
     AND       
        i.name IS NOT NULL AND s.database_id = DB_ID() 
'	

SELECT 
    ServerName
    ,DatabaseName
    ,ReportDate
    ,COALESCE(user_seeks,0) AS user_seeks
    ,COALESCE(user_scans,0) AS user_scans
    ,COALESCE(user_lookups,0) AS user_lookups
    ,COALESCE(reads,0) AS reads
    ,COALESCE(writes,0) AS writes
    ,ServerStartDate
 FROM #DatabaseAccess




# Job steps snippet
> ## Easy way to see Jobsteps with blank lines added to aid clarity

In [ ]:

WITH CTE_JobSteps
AS
(
SELECT sj.Name,sj.[description],sjs.step_name,sjs.step_id,sjs.command,sjs.subsystem,sjs.database_name, ROW_NUMBER() OVER (Partition by sj.Name  Order By sj.Name,sjs.step_id ) AS RowID, 1 AS Show
FROM msdb.dbo.sysjobs sj
INNER JOIN msdb.dbo.sysjobsteps sjs ON sjs.job_id =sj.job_id
WHERE sj.enabled =1
UNION ALL
SELECT sj.Name ,'' AS description,'' AS step_name,'' AS step_id,'' AS command,'' AS subsystem,'' AS database_name,MAX(step_id)+1 AS RowID,0 AS Show
FROM msdb.dbo.sysjobs sj
INNER JOIN msdb.dbo.sysjobsteps sjs ON sjs.job_id =sj.job_id
WHERE sj.enabled =1
GROUP BY sj.name
)
SELECT 
    CASE WHEN show =1 THEN  Name ELSE '' END AS Name
    ,CASE WHEN show =1 THEN  CAST(Step_id AS CHAR(4)) ELSE '' END AS Step_id
    ,step_name
    ,command
    ,subsystem
    ,database_name
FROM CTE_JobSteps c
ORDER BY c.Name,c.RowID

## Converting sp_execute 
> ### (Shift+Ctrl+P) and search for convert

# Formatting SQL
>## ctrl+shift+ P type poor SQL or set a hotkey (Shift-I) (Extension)

In [ ]:
-- List columns in all tables whose name is like 'TableName'
SELECT 
    TableName = tbl.table_schema + '.' + tbl.table_name, 
    ColumnName = col.column_name, 
    ColumnDataType = col.data_type
FROM INFORMATION_SCHEMA.TABLES tbl
INNER JOIN INFORMATION_SCHEMA.COLUMNS col 
    ON col.table_name = tbl.table_name
    AND col.table_schema = tbl.table_schema

WHERE tbl.table_type = 'base table' and tbl.table_name like '%TableName%'
GO
SELECT   e.[BusinessEntityID],p.[Title],p.[FirstName],p.[MiddleName] ,p.[LastName],p.[Suffix],e.[JobTitle]  ,pp.[PhoneNumber]
    ,pnt.[Name] AS [PhoneNumberType],ea.[EmailAddress] ,p.[EmailPromotion]
    ,a.[AddressLine1] ,a.[AddressLine2],a.[City] ,sp.[Name] AS [StateProvinceName] 
    ,a.[PostalCode],cr.[Name] AS [CountryRegionName]   ,p.[AdditionalContactInfo]
FROM [HumanResources].[Employee] e 	INNER JOIN [Person].[Person] p	ON p.[BusinessEntityID] = e.[BusinessEntityID]
    INNER JOIN [Person].[BusinessEntityAddress] bea     ON bea.[BusinessEntityID] = e.[BusinessEntityID]     INNER JOIN [Person].[Address] a 
    ON a.[AddressID] = bea.[AddressID]    INNER JOIN [Person].[StateProvince] sp 
    ON sp.[StateProvinceID] = a.[StateProvinceID]    INNER JOIN [Person].[CountryRegion] cr 
    ON cr.[CountryRegionCode] = sp.[CountryRegionCode]   
 LEFT OUTER JOIN [Person].[PersonPhone] pp
ON pp.BusinessEntityID = p.[BusinessEntityID]    
LEFT OUTER JOIN [Person].[PhoneNumberType] pnt
    ON pp.[PhoneNumberTypeID] = pnt.[PhoneNumberTypeID]
    LEFT OUTER JOIN [Person].[EmailAddress] ea
    ON p.[BusinessEntityID] = ea.[BusinessEntityID];
GO


# Generating SQL Scripts
> ## Ability to script a table and specify where clause (extention)
1. Install extension
2. Go to server and drillk through to table and right click select script as

# Running a Query you've aready ran
> ## Ability execute previously ran code (extention)
1. Install extension
2. Type ctrl-j and select Query History